# Describe a PIV Result File

In this example, we want to describe the result file of the ILA Vortex pair (source: https://www.pivtec.com/download/samples/VortexPairSeq.zip)

Where to start? Let's first evaluate what we have:

General context:
- The general concept of a dataset is described by `dcat:Dataset`
- The file is described by `pivmeta:PivResultDistribution` and is part of the `dcat:Dataset`

Specific information:<br>
Of greater interest is the PIV process including the PIV parameters leading to the dataset. A `dcat:Dataset` is the output of the PIV process. For this, we can use [`m4i:output of`](http://purl.obolibrary.org/obo/RO_0002353):

In [1]:
from pivmetalib import pivmeta, dcat, m4i, schema, sd, prov

In [2]:
result_dist = pivmeta.PivResultDistribution(
    title='Result File',
    download_URL='file:///vp1a.dat'
)
# the "downloaded" file must exist:
assert result_dist.download().exists()

In [3]:
pivmeta.PivImageType.ExperimentalImage

<PivImageType.ExperimentalImage: 'https://matthiasprobst.github.io/pivmeta#ExperimentalImage'>

In [4]:
piv_image_dist = pivmeta.PivImageDistribution(
    title='ILA Vortex Pair Images',
    download_URL='https://www.pivtec.com/download/samples/pivimg1.zip',
    piv_image_type=pivmeta.PivImageType.ExperimentalImage
)
piv_image_dist.piv_image_type

Url('https://matthiasprobst.github.io/pivmeta#ExperimentalImage')

In [5]:
ds = dcat.Dataset(
    title='ILA Vortex Pair',
    distribution=[result_dist, piv_image_dist]
)
ds

Dataset(title=ILA Vortex Pair, distribution=[PivResultDistribution(title=Result File, download_URL=file:///vp1a.dat, ), PivImageDistribution(title=ILA Vortex Pair Images, download_URL=https://www.pivtec.com/download/samples/pivimg1.zip, piv_image_type=https://matthiasprobst.github.io/pivmeta#ExperimentalImage, )], )

Before defining the PIV processing steps, we need to describe the software used:

In [6]:
piv_software = pivmeta.PIVSoftware(
    author=prov.Organisation(
        name='PIVTEC GmbH',
        mbox='info@pivtec.com',
        url='https://www.pivtec.com/'
    ),
    has_documentation='https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf',
)

In [7]:
import h5py
from typing import Dict


import h5rdmtoolbox as h5tbx
h5tbx.set_config(natural_naming=False)

In [8]:
with h5tbx.File() as h5:
    piv_software.dump_hdf(h5)
    h5.dump()

In [9]:
print(piv_software.dump_jsonld())

{
    "@context": {
        "@import": "https://raw.githubusercontent.com/matthiasprobst/pivmeta/main/pivmeta_context.jsonld"
    },
    "@graph": [
        {
            "@id": "https://local-domain.org/f1387bc4-1b67-4931-a3dc-44d2ac5b2d83",
            "@type": "prov:Organisation",
            "foaf:mbox": "info@pivtec.com",
            "foaf:name": "PIVTEC GmbH",
            "url": "https://www.pivtec.com/"
        },
        {
            "@id": "https://local-domain.org/af7bfd2f-b8c2-455f-9afb-4ad77d9651de",
            "@type": "Piv Software",
            "schema:author": {
                "@id": "https://local-domain.org/f1387bc4-1b67-4931-a3dc-44d2ac5b2d83"
            },
            "sd:hasDocumentation": "https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf"
        }
    ]
}


## Processing steps

A `PivProcessingStep` class is provided in order to distinguish the processing step from others. Some methods are provided as classes but without specific properties. This is done to provide flexibility, as all methods can be standardized. However, by introducing **standard names**, the authors may narrow their parameter definitions either by using global standard names (with an IRI) or within their project.

*TODO: Put here an image illustrating the possibilities*

By using standard names, important parameters can be identified unambiguously. See `PIVMETA.image_filter_kernel_size` in the example in contrast to the 180° image rotation.

### 1. Pre-Processing (Image processing)

Methods:
- image rotation by 180 deg

In [10]:
from pivmetalib.namespace import PIVMETA  # Or define a standard name table here...

In [11]:
pre = pivmeta.PivPreProcessing(
    name='Image pre processing',
    realizes_method=[
        m4i.Method(
            description='Rotates the input image by 180 deg',
            has_parameter=m4i.NumericalVariable(
                name='rotation',
                has_numerical_value=180,
                has_unit='deg',
                kind_of_quantity='https://qudt.org/vocab/unit/DEG'
            )
        ),
        # Dont define all the classes for filters and outlier detection because everybody may define it differently.
        # common parameters can be specified "on demand" by standard names like so: 
        m4i.Method(
            description='Applies a median filter to the image',
            has_parameter=pivmeta.NumericalVariable(
                standard_name=PIVMETA.image_filter_kernel_size,
                has_numerical_value=3,
            )
        )
    ]
)
pre

PivPreProcessing(realizes_method=[Method(description=Rotates the input image by 180 deg, has_parameter=id=None label=None has_variable_description=None has_symbol=None has_numerical_value=180 has_unit='deg' has_kind_of_quantity=None name='rotation' kind_of_quantity='https://qudt.org/vocab/unit/DEG', ), Method(description=Applies a median filter to the image, has_parameter=id=None label=None has_variable_description=None has_symbol=None has_numerical_value=3 has_unit=None has_kind_of_quantity=None standard_name=Url('https://matthiasprobst.github.io/pivmeta#image_filter_kernel_size'), )], name=Image pre processing)

## 2. PIV evaluation

The PIV evaluation is a `m4i:ProcessingStep`. It realizes the methods
- `pivemta:CorrelationMethod` (e.g. FFT, ... ),
- `pivemta:InterrogationMethod` (e.g. `pivemta:Multigrid`, ...) and
- `pivemta:PeakSearchMethod`.

The outlier detection (validation) is a sub-processing step (`pivemta:PivValidation`), because it can realize multiple methods. Most popular ones are described by classes:
- `pivemta:DynMean`.

Here, a multigrid evaluation is performed using standard FFT.

### 2.1 Correlation algorithm

The correlation algorithm is a subclass of `m4i:Method`. At least a name and description should be provided (here, taken from the documentation). We could also provide parameters.

In [12]:
calgo = pivmeta.CorrelationAlgorithm(
    name='Standard (FFT) Correlation',
    description='The default mode of cross-correlation using FFTs to speed the computation. ' 
    'In principle the sum of pixel-wise multiplication of intensities is computed for each ' 
    'correlation offset (For implementation details please refer to Raffel et al. 2007).'
)
calgo

CorrelationAlgorithm(description=The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007)., name=Standard (FFT) Correlation)

### 2.2 Interrogation method

In this example, a *Multi-Grid* method was used starting from a window with size 64 px down to 16 px in 3 steps

In [13]:
v = pivmeta.NumericalVariable(
            has_standard_name=PIVMETA.x_initial_interrogation_window_size,
            has_numerical_value=64,
        )

In [14]:
#print(v.dump_jsonld())

In [15]:
int_meth = pivmeta.Multigrid(
    name='Multigrid interrogation method',
    description='Run a multigrid PIV algorithm on all images',
    has_parameter=[
        pivmeta.NumericalVariable(
            has_standard_name=PIVMETA.x_initial_interrogation_window_size,
            has_numerical_value=64,
        ),
        pivmeta.NumericalVariable(
            has_standard_name=PIVMETA.y_initial_interrogation_window_size,
            has_numerical_value=64,
        ),
        pivmeta.NumericalVariable(
            has_standard_name=PIVMETA.x_final_interrogation_window_size,
            has_numerical_value=16,
        ),
        pivmeta.NumericalVariable(
            has_standard_name=PIVMETA.y_final_interrogation_window_size,
            has_numerical_value=16,
        ),
        pivmeta.NumericalVariable(
            has_standard_name=PIVMETA.x_final_interrogation_window_overlap_size,
            has_numerical_value=8,
        ),
        pivmeta.NumericalVariable(
            has_standard_name=PIVMETA.y_final_interrogation_window_overlap_size,
            has_numerical_value=8,
        ),
        pivmeta.NumericalVariable(
            has_standard_name=PIVMETA.number_of_multigrid_passes,
            has_numerical_value=3,
        )
    ]
)
int_meth

AttributeError: module 'pivmetalib.pivmeta' has no attribute 'Multigrid'

### 2.3 Outlier detection

We use the following two methods for outlier detection:
- normalized median test threshold: 3.0 (see DOI=https://doi.org/10.1007/s00348-005-0016-6)
- dynamic mean test: mean=2.0, var=1.0

In [ ]:
median_test = pivmeta.OutlierDetectionMethod(
    name='normalized median test',
    has_parameter=m4i.NumericalVariable(
        name='threshold',
        has_numerical_value=3.0
    )
)

In [ ]:
dyn_mean = pivmeta.OutlierDetectionMethod(
    name='dynamic mean test',
    has_parameter=[
        m4i.NumericalVariable(
            name='mean',
            has_numerical_value=2.0
        ),
        m4i.NumericalVariable(
            name='var',
            has_numerical_value=1.0
        )
    ]
)

In [ ]:
proc = pivmeta.PivEvaluation(
    name='piv evaluation',
    realizes_method=[
        calgo,
        int_meth,
        median_test,
        dyn_mean
    ]
)
proc

In [ ]:
data_smoothing = m4i.Method(
    name='Low-pass filtering',
    description='applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.',
    has_parameter=m4i.NumericalVariable(name='kernel', has_numerical_value=2.0)
)

In [ ]:
post = pivmeta.PivPostProcessing(
    name='Post processing',
    realizes_method=data_smoothing
)
            

## 3. Creating the full Meta document (connect information)

We created three processing steps:
1. pre (takes raw images)
2. proc
3. post (outputs result data)


In [ ]:
post.has_output = ds

## 4. dump piv run to JSON-LD

In [ ]:
piv = m4i.ProcessingStep(
    name='PIV Run',
    starts_with=pre,
    ends_with=post
)
# proc.part_of = piv
pre.precedes = proc
proc.precedes = post

# all processing steps were employed by software pivview:
pre.has_employed_tool = pivtec
proc.has_employed_tool = pivtec
post.has_employed_tool = pivtec

In [ ]:
with open('piv_process.json', 'w') as f:
    f.write(proc.dump_jsonld())

In [ ]:
from pivmetalib.jsonld import dump_jsonld

In [ ]:
print(dump_jsonld([pre, proc]))

# Query JSON-LD file

t.b.d